# N-Gram Language modeling

> "old school" language modeling based on counting tokens in data

---
skip_exec: true
skip_showdoc: true
---

In [ ]:
#| default_exp models.ngram

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export
import torch.nn as nn
import torch
import torch.nn.functional as F

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from typing import Dict, List, Tuple, Optional, Set
from collections import Counter

import kenlm


https://github.com/karpathy/makemore

## Unigram

In [ ]:
#| export
class CharUnigram:
    def __init__(self, data:List[str]):
        self._count = {}
        self.total_count = 0
        self.unique_chars = set()
        for name in data:
            for c in name:
                self.unique_chars.update(c)
                if c in self._count:
                    self._count[c] += 1
                else:
                    self._count[c] = 1
                self.total_count += 1
        self._probs = {k:v/self.total_count for k,v in self._count.items()}
        self._count = self.sort_dict_by_value(self._count)
        self._probs = self.sort_dict_by_value(self._probs)
        self.unique_chars = sorted(self.unique_chars)
        self._stoi = {v:idx for idx,v in enumerate(self.unique_chars)}
        self._itos = {idx:v for idx,v in enumerate(self.unique_chars)}
        
    def __len__(self):
        return len(self.unique_chars)
    
    @staticmethod
    def sort_dict_by_value(dict:Dict, reverse:bool=True)->Dict:
        return {k:v for k,v in sorted(dict.items(), reverse=reverse, key=lambda x:x[1])}

    @property
    def counts(self)->Dict:
        return(self._count)

    @property
    def probs(self)->Dict:
        return(self._probs)

    @property
    def chars(self)->List:
        return self.unique_chars
        
    def stoi(self, char:str)->int:
        return(self._stoi[char])
    
    def itos(self, idx:int)->str:
        return(self._itos[idx])
    
    def sample(self)->str:
        # get probs for order list of characters to build prob table
        prob_distrib = torch.tensor([self._probs[k] for k in self.unique_chars])
        idx = int(torch.multinomial(prob_distrib,num_samples=1,replacement=True))
        return(self._itos[idx])
        

### Usage

In [ ]:
# without pandas
with open('../data/text/names.txt', 'r') as f:
    list_of_words = f.read().splitlines()
# with pandas
df = pd.read_csv('../data/text/names.txt', names=['name'], header=None)
list_of_words = list(df.head().name)

unigram = CharUnigram(list_of_words)
print("sorted counts: ", unigram.counts)
print("sorted probs: ", unigram.probs)
print(len(unigram))
print(unigram.chars)
print(unigram._stoi)
print(unigram.stoi('a'))
print(unigram.itos(0))

In [ ]:
df = pd.DataFrame.from_dict(unigram.counts, orient='index')
df.plot(kind='bar')

In [ ]:
samples = []
for i in range(10000):
    s = unigram.sample()
    samples.append(s)

# sampled
count = Counter([c for w in samples for c in w])
df = pd.DataFrame.from_dict(count, orient='index')
df[0].sort_values(ascending=False).plot(kind='bar')

## Bigram

In [ ]:
class CharBigram():
    def __init__(self):
        pass
    

### Usage

In [ ]:
# data
with open('../data/text/names.txt', 'r') as f:
    data = f.read().splitlines()
print("first lines of text: ", data[:10])

# data = ["this is a text"]

In [ ]:
# bigram counts
bigrams = {}
unique_tokens = set()
for name in data:
    line = list(name)
    unique_tokens.update(line)
    line.append('<stop>')
    line.insert(0, '<stop>')
    for i,v in enumerate(range(len(line)-1)):
        bigram = (line[i], line[i+1])
        if bigram in bigrams:
            bigrams[bigram] += 1
        else:
            bigrams[bigram] = 1

# print("unsorted: ", list(bigrams)[:10])
# print("sorted: ", sort_dict_by_value(bigrams))


### Numericalization

In [ ]:
tokens = sorted(unique_tokens)
# use same for start & stop in this case (separate lines of names)
# tokens.append('<start>')
tokens.append('<stop>')
print(tokens)
stoi = {v:i for i,v in enumerate(tokens)}
itos = {i:v for i, v in enumerate(tokens)}
print(stoi, itos)

### Matrix representation

In [ ]:
n_toks = len(tokens)
print(n_toks)
N = torch.zeros((n_toks, n_toks)).long()
print(N.shape)

In [ ]:
for bigram, value in bigrams.items():
    idx1, idx2 = stoi[bigram[0]], stoi[bigram[1]]
    N[idx1, idx2] = value

plt.xlabel('char_t+1')
plt.ylabel('char_t')
i = [i for i, v in itos.items()]
v = [v for i,v in itos.items()]
plt.xticks(i, v)
plt.yticks(i, v)
plt.imshow(N, origin='lower')

### From counts to probabilities

In [ ]:
print(N)

In [ ]:
# smoothing avoids having log(0) = inf when computing NLL loss
smoothing = 1
P = (N.float()+smoothing) / N.sum(1,keepdim=True)
plt.imshow(P, origin='lower')

In [ ]:
row_6 = (N[6,:]/N[6,:].sum())
print(row_6)
print(row_6.sum())

In [ ]:
p = P[6, :]
print(p.sum(), p.max(), torch.argmax(p))


### Sampling

In [ ]:
for i in range(10):
    res = []
    prev = stoi['<stop>']
    while True:
        # max prob sampling
        next = int(torch.argmax(P[prev, :]))
        # multinomial sampling
        next = int(torch.multinomial(P[prev,:],num_samples=1,replacement=True))
        if next == stoi['<stop>']:
            print(''.join(res))
            break
        else:
            res.append(itos[next])
            prev = next


### Log likelihood loss function

In [ ]:
bigram_p = {}
for bigram, value in bigrams.items():
    idx1, idx2 = stoi[bigram[0]], stoi[bigram[1]]
    bigram_p[bigram] = P[idx1,idx2]

print(bigram_p)

In [ ]:
bigram_p_sorted = {k: v.float() for k, v in sorted(bigram_p.items(), reverse=True, key=lambda x: x[1])}
print(bigram_p_sorted)

In [ ]:
# likelihood of full corpus = product of all bigram prods
l = 0
for bigram, prob in bigram_p_sorted.items():
    l += torch.log(prob)

# negative log likelihood loss nll
nll = -l /len(bigram_p_sorted)
print(nll)

### Generate training data

In [ ]:
word = "this"
sample = [(word[i], word[i+1]) for i,c in enumerate(word) if i < len(word)-1]
print(list(zip(*sample)))

In [ ]:
xs, ys = [], []
for word in data:
    sample = [(stoi[word[i]], stoi[word[i+1]]) for i,c in enumerate(word) if i < len(word)-1]
    x, y = list(zip(*sample)) # inverse of zip
    xs.append(torch.tensor(x))
    ys.append(torch.tensor(y))

In [ ]:
print('x:', xs[:3])
print('y', ys[:3])

### 1-hot encoded input

In [ ]:
enc = [F.one_hot(x, num_classes=len(tokens)).float() for x in xs]
print(enc[:3])

In [ ]:
plt.imshow(enc[0])

In [ ]:
X = enc[0]
print(X.shape)

### 'Neural net' modeling
we model the transition probability matrix by neural net activations

In [ ]:
W = torch.randn(27, 27)

In [ ]:
logits = X @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
print(probs)

## KenLM
We refer to efficient kenlm implementation for larger n-gram models usable for production

In [ ]:
#| export 

class KenLM:
    def __init__(self, arpa_path:str, vocab:List):
        # TODO: deal with zipped arpa models
        self.model = kenlm.LanguageModel(arpa_path)
        self.partial_text = []
        self.partial_score = 0.0
        # init new sentence
        self.s1 = kenlm.State()
        self.s2 = kenlm.State()
        self.model.BeginSentenceWrite(self.s1)
        self.vocab = vocab

    def init_vocab(self, vocab_path):
        with open(vocab_path) as f: self.vocab = f.read().splitlines()
    
    def new_sentence_init(self):
        self.partial_text = []
        self.partial_score = 0.0
        self.s1 = kenlm.State()
        self.s2 = kenlm.State()
        self.model.BeginSentenceWrite(self.s1)
    
    def append(self, word:str):
        # add word to beam and update probs
        if word == '.':
            self.partial_score += self.model.BaseScore(self.s1, word, self.s2)
            self.partial_score += self.model.BaseScore(self.s2, '</s>', self.s1)
            self.partial_text.append(word)
        else:
            self.partial_score += self.model.BaseScore(self.s1, word, self.s2)
            # input <=> output state
            self.s1, self.s2 = self.s2, self.s1
            self.partial_text.append(word)
    
    def peek(self, word:str, log_prob:bool=True)->float:
        if log_prob:
            # check prob of next word given context without update state
            res = self.partial_score+self.model.BaseScore(self.s1, word, self.s2)
        else:
            res = 10**(self.partial_score+self.model.BaseScore(self.s1, word, self.s2))
        return(res)

    def sentence_score(self, sentence:str)->float:
        return(self.model.score(sentence))

    def partial_sentence_score(self, sentence:str)->float:
        return(self.model.score(sentence, eos=False))
        
    def nbest(self, n:int, log_prob:bool=True):
        if not self.vocab:
            print('need to init vocab')
        res = []
        for word in self.vocab:
            res.append((word, self.peek(word, log_prob)))

        return sorted(res, key=lambda x: x[1], reverse=True)[:n]

    @property
    def score(self)->float:
        return(self.partial_score)

    @property
    def text(self)->str:
        return(' '.join(self.partial_text))
    

### Preprocess data into kenlm format
tokens separated by space with new sentence at each line

In [ ]:
df = pd.read_csv('../data/text/names.txt', header=None, names=['name']) 
df = df.name.apply(lambda x: list(x)) # str into list of char
# df.apply(lambda x: x.append('<eos>')) # if eos needed
print(df.head())
df_toks = df.str.join(' ') # for kenlm input format tokens are separated by space
print(df_toks.head())


#### Unique tokens

In [ ]:
df.head()
# for row in df.iterrows():
#     print(row)
tokens = set()
for k,v in df.items():
    tokens.update(list(v))

print(tokens)
len(tokens)

#### Save data to kenlm format for training

In [ ]:
data_file = df.to_csv('../data/text/names.kenlm.txt', header=None, index=None)
! bzip2 -kz ../data/text/names.kenlm.txt
! bzcat ../data/text/names.kenlm.txt.bz2 | head

### Train KenLM n-gram model
https://lukesalamone.github.io/posts/running-simple-language-model/

KenLM requires data to be one sentence per line lowercase

In [ ]:
# ! if [ ! -f "../data/text/names.2gram.arpa" ]; then lmplz --discount_fallback -o 2 < ../data/text/names.kenlm.txt.bz2>../data/text/names.2gram.arpa; fi

In [ ]:
# ! if [ ! -f "../data/text/names.2gram.kenlm" ]; then build_binary ../data/text/names.2gram.arpa ../data/text/names.2gram.kenlm; fi

#### Test original Kenlm python api probs

In [ ]:
model = kenlm.LanguageModel('../data/text/names.2gram.kenlm')
sentence = "emma"
tokenized = "e m m a"
# model.score("emma", bos = False, eos = False)
words = ['<s>'] + list(sentence) + ['</s>']
print(words)
final = 0
for i, (prob, length, oov) in enumerate(model.full_scores(tokenized)):
    print(f'words: {words[i:i+length]} index:{i}, prob:{prob}, length:{length}, oov:{oov}')
    final += prob

print(final)
print(model.score("e m m a"))
print(f'prob <s> e: {model.score("e", bos=True, eos=False)}')
print(f'prob e: {model.score("e", bos=False, eos=False)}')
print(f'prob <s> e m: {model.score("e m", bos=True, eos=False)}')
print(f'prob e m: {model.score("e m", bos=False, eos=False)}')
state = kenlm.State()
state2 = kenlm.State()
model.BeginSentenceWrite(state)
accum = 0
accum += model.BaseScore(state, "e", state2)
print(f'prob <s> e: {accum}')
state, state2 = state2, state
accum += model.BaseScore(state, "m", state2)
print(f'prob <s> e m: {accum}')


#### Define LM vocabulary

In [ ]:
# add special tokens to vocabulary
tokens.add('<s>')
tokens.add('</s>')
tokens.add('<unk>')
print(tokens, len(tokens))
vocab = list(tokens)

### Inference / Sampling from prob distributions

In [ ]:
lm = KenLM('../data/text/names.2gram.kenlm', vocab)
init_char = '<s> e m m'
# probs = lm.nbest(len(vocab), log_prob=False)
# print(np.sum([p for char, p in probs]))
# res = [init_char]
# next = int(torch.multinomial(P[prev,:],num_samples=1,replacement=True))
for i in range(50):
    lm.new_sentence_init()
    lm.append(init_char)
    while True:
        # nbest probs at current state
        probs = lm.nbest(len(vocab), log_prob=False)
        # print(probs)
        # print(np.sum(probs))
        # sample from prob distribution
        try:
            index_next = int(torch.multinomial(torch.tensor([prob for char, prob in probs]),num_samples=1,replacement=True))
        except:
            print("probs too small")
            break
        char_next = probs[index_next][0]
        lm.append(char_next)
        # print(init_char + '<s>')
        if char_next == '</s>' or char_next == '<s>' and lm.text != init_char and (lm.text != init_char+' <s>'):
            print(lm.text.replace(' ', ''))
            break



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()